This notebook contains K-Fold cross validation with the entire dataset (across all subjects).
The dataset needs to be preprocessed using **data_preprocessing.ipynb** first

In [1]:
# Necessary imports
import os
import numpy as np
import tensorflow as tf
import keras
import nengo_dl
import random
from tensorflow.python.keras import Input, Model
from tensorflow.python.keras.layers import Conv2D, Dropout, AveragePooling2D, Flatten, Dense, BatchNormalization, Conv3D
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from keras import backend as K

In [2]:
dataset_path = os.path.join('dataset_result', 'bci_dataset.npz')
dataset = np.load(dataset_path)
features, labels = dataset['features'], dataset['labels'] # get features and labels

# Additional variables to reproduce the same result
seed = 1
np.random.seed(seed)
tf.random.set_seed(seed)

The dataset needs to be reshaped in order to be use it with Nengo framework.
We flatten features and add time dimension to them, labels are encoded using one-hot encoding
(plus additional time dimension is added)

In [3]:
features = features.reshape((features.shape[0], 1, -1)) # flatten and add time dimension

labels = labels.reshape(-1, 1)
labels = OneHotEncoder().fit_transform(labels).toarray()
labels = labels.reshape((labels.shape[0], 1, -1)) # add time dimension as well

print('Features shape: ', features.shape)
print('Labels shape: ', labels.shape)

Features shape:  (2976, 1, 5040)
Labels shape:  (2976, 1, 2)


Define a tensorflow model which will be used with the dataset


In [4]:
# CNN model creator function
def cnn_model_1():

    inp = Input(shape=(14, 360, 1), name='input_layer')
    conv2d = Conv2D(filters=9, kernel_size=(3, 3), activation='relu')(inp)
    dropout1 = Dropout(0.5, seed=seed)(conv2d)
    avg_pooling = AveragePooling2D(pool_size=(2, 2))(dropout1)
    flatten = Flatten()(avg_pooling)
    dense1 = Dense(1000, activation='relu')(flatten)
    batch_norm = BatchNormalization()(dense1)
    dense2 = Dense(500, activation='relu')(batch_norm)
    dropout2 = Dropout(0.5, seed=seed)(dense2)
    output = Dense(2, activation='softmax', name='output_layer')(dropout2)

    return Model(inputs=inp, outputs=output)

def dense_only_model():
    inp = Input(shape=(14, 360, 1), name='input_layer')
    flatten = Flatten()(inp)
    dense1 = Dense(1260, activation='relu')(flatten)
    dense2 = Dense(630, activation='relu')(dense1)
    dense3 = Dense(315, activation='relu')(dense2)
    dense4 = Dense(15, activation='relu')(dense3)
    output = Dense(2, activation='sigmoid', name='output_layer')(dense4)

    return Model(inputs=inp, outputs=output)

In [5]:
def run_ann(model, x_train, y_train, x_test, y_test, params_save_path, iteration, epochs=30):
    converter = nengo_dl.Converter(model)

    with nengo_dl.Simulator(converter.net, minibatch_size=10) as simulator:
        simulator.compile(
            optimizer=keras.optimizers.Adam(),
            loss=keras.losses.BinaryCrossentropy(),
            metrics=['accuracy']
        )

        # Train the model and save weight params
        simulator.fit(x=x_train, y=y_train, epochs=epochs)
        simulator.save_params(params_save_path)

        # Test trained model and print result
        eval = simulator.evaluate(x=x_test, y=y_test)
        print('{}. ANN accuracy: {:5f}%'.format(
            iteration, eval['probe_accuracy'] * 100
        ))

        return eval['probe_accuracy'] # return accuracy


In [6]:
# Code for SNN
def run_snn():
    pass


In [7]:
# Shuffle data
indices = [i for i in range(features.shape[0])]
random.shuffle(indices)
features = features[indices, :, :]
labels = labels[indices, :, :]

print(features.shape)
print(labels.shape)

(2976, 1, 5040)
(2976, 1, 2)


In [8]:
splits = 10
iter = 0

ann_results = []
params_folder = 'loo_concat_output'
os.makedirs(params_folder, exist_ok=True) # create a directory if it does not exist
for train_idx, test_idx in KFold(splits).split(features):

    # Get training and testing data
    x_train, x_test = features[train_idx], features[test_idx]
    y_train, y_test = labels[train_idx], labels[test_idx]

    print('X_train shape:', x_train.shape, 'X_test shape:', x_test.shape)
    print('Y_train shape:', y_train.shape, 'Y_test shape:', y_test.shape)

    model = dense_only_model() # instantiate network model

    # Run ANN and save the result
    ann_results.append(
        run_ann(
        model, x_train, y_train, x_test, y_test,
        params_save_path=os.path.join(params_folder, 'params_{}'.format(iter)),
        iteration=iter
        )
    )

    iter += 1
    K.clear_session() # clear session otherwise system may run out of resources



X_train shape: (2678, 1, 5040) X_test shape: (298, 1, 5040)
Y_train shape: (2678, 1, 2) Y_test shape: (298, 1, 2)
Build finished in 0:00:00                                                      
Optimization finished in 0:00:00                                               
Construction finished in 0:00:00                                               
Epoch 1/30
267/267 [==============================] - 11s 31ms/step - loss: 0.7838 - probe_loss: 0.7838 - probe_accuracy: 0.5166

c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\simulator.py:460: UserWarning: No GPU support detected. See https://www.nengo.ai/nengo-dl/installation.html#installing-tensorflow for instructions on setting up TensorFlow with GPU support.
  warnings.warn(
c:\dev\anaconda\envs\spiking-env\lib\site-packages\nengo_dl\simulator.py:1921: UserWarning: Number of elements in input data (2678) is not evenly divisible by Simulator.minibatch_size (10); input data will be truncated.
  warnings.warn(


KeyboardInterrupt: 